# HERRAMIENTAS PARA PARALELIZAR

### ¿Cuántos núcleos tengo?

In [1]:
import multiprocessing as mp
print("Number of processors: ", int(mp.cpu_count()/2))

Number of processors:  2


### HERRAMIENTAS

**PAQUETE MULTIPROCESSING** 

Dento del paquete multiproccesing existen dos procesos para paralelizar :

* Pool
* Process 

Pool y Process ejecutan la tarea en paralelo, pero su forma de ejecutar la tarea en paralelo es diferente. A continuación se realiza una prueba con datos sintéticos para entender el funcionamiento de Pool.

POOL

In [2]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    p = Pool(processes=4) 
    print(p.map(f, [1, 2, 3]))

[1, 4, 9]


Dentro de Pool tenemos tres opciones:
* **.map()**: map solo puede tomar un iterable (lista,tupla,etc) como argumento. Es más adecuado para operaciones iterables más simples pero hace el trabajo más rápido. La función map () devuelve un elemento iterable (lista, tupla, etc) de los resultados después de aplicar una función dada a cada posición de otro elemento iterable.
* **.apply()**: toma un argumento que acepta los parámetros pasados a la función que queremos paralelizar como argumento
* **.starmap()**: es como una versión de Pool.map () que acepta argumentos.

In [3]:
# Python program to demonstrate working of map. 
  
# Return double of n 
def addition(n): # FUNCIÓN
    return n + n 
  
# We double all numbers using map() 
numbers = (1, 2, 3, 4) # ITERABLE
result = map(addition, numbers) 
print(list(result)) 

[2, 4, 6, 8]


**PAQUETE NUMBA**

A continuación, se muestra un ejemplo sencillo de paralelización con Numba. Sin embargo, para el proyecto no se ha aplicado este proceso de paralelización debido a que en gran parte de los procesos se hace uso del paquete Pandas.

In [4]:
from numba import jit
import numpy as np

x = np.arange(100).reshape(10, 10)

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

print(go_fast(x))

[[  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.]
 [ 29.  30.  31.  32.  33.  34.  35.  36.  37.  38.]
 [ 39.  40.  41.  42.  43.  44.  45.  46.  47.  48.]
 [ 49.  50.  51.  52.  53.  54.  55.  56.  57.  58.]
 [ 59.  60.  61.  62.  63.  64.  65.  66.  67.  68.]
 [ 69.  70.  71.  72.  73.  74.  75.  76.  77.  78.]
 [ 79.  80.  81.  82.  83.  84.  85.  86.  87.  88.]
 [ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.]
 [ 99. 100. 101. 102. 103. 104. 105. 106. 107. 108.]]


**CYTHON**

Tras instalar la extension, se muestra un ejemplo sintético de funciones en Python aplicadas sobre un dataframe. En primer lugar, se muestra este proceso sin utilizar Cython y se observa el tiempo que tarda en realizarse, y por último, utilizando la extensión para ver la diferencia. 

In [5]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'a': np.random.randn(1000),
                    'b': np.random.randn(1000),
                    'N': np.random.randint(100, 1000, (1000)),
                    'x': 'x'})

SIN LA EXTENSIÓN

In [6]:
def f(x):
    return x * (x - 1)

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Tiempo que tarda:

In [7]:
%timeit df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

171 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


CON LA EXTENSIÓN

In [8]:
%load_ext Cython

In [9]:
 %%cython
def f_plain(x):
    return x * (x - 1)
def integrate_f_plain(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx

In [10]:
%timeit df.apply(lambda x: integrate_f_plain(x['a'], x['b'], x['N']), axis=1)

102 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**GOOGLE COLAB**

Las pruebas realizadas con esta plataforma, se adjuntan como anexo en la memoria.